<a href="https://colab.research.google.com/github/supplient/bachelor_design/blob/bert_replace/Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Setup

In [6]:
!pip install keras_bert

  Created wheel for keras-bert: filename=keras_bert-0.81.0-cp36-none-any.whl size=37913 sha256=7939f9569f546363689cd7138c2de677831a657ce0b9cb17b4a22d35ae280552
  Stored in directory: /root/.cache/pip/wheels/bd/27/da/ffc2d573aa48b87440ec4f98bc7c992e3a2d899edb2d22ef9e
  Created wheel for keras-transformer: filename=keras_transformer-0.32.0-cp36-none-any.whl size=13266 sha256=14bcb23b40f8ca9ad2dcadcca574cd864019860993a229f1d3ee0d30066e6464
  Stored in directory: /root/.cache/pip/wheels/62/f0/ce/82fa5d024d5ef8e263f26a50dcee23820efe245680ce9c922a
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7554 sha256=5f175c46fe426e257956196b1d27a62c65b9d013f661b07ad84e1f79de2bef7a
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.22.0-cp36-none-any.whl size=15371 sha256=7d7b17ed39c081f67b36c00bd2cd4ce489ea3f456e9c3b4bea79882910754d58
  Stor

In [1]:
!git clone https://github.com/supplient/bachelor_design.git
import os
os.chdir("bachelor_design")
!pwd

Cloning into 'bachelor_design'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 88 (delta 35), reused 64 (delta 18), pack-reused 0
Unpacking objects: 100% (88/88), done.
/content/bachelor_design


In [2]:
!git checkout bert_replace
!git pull

Branch 'bert_replace' set up to track remote branch 'bert_replace' from 'origin'.
Switched to a new branch 'bert_replace'
Already up to date.


In [2]:
from driver_amount import addh

[Locale] Using address head: /mnt/d/My Drive


# Prepare Input

In [1]:
import config

In [7]:
import preprocess

Using TensorFlow backend.


In [0]:
import pickle
tag_vocab = None
with open(addh + config.TAG_VOCAB_PATH, "rb") as fd:
    tag_vocab = pickle.load(fd)

In [0]:
SEQ_LEN = 512

test_char_seqs, test_tag_seqs = preprocess.load_file(
    addh + config.TEST_DATA_PATH)
test_token_id_seqs, test_segment_seqs, test_tag_id_seqs, tag_vocab = preprocess.preprocess(
    test_char_seqs, 
    test_tag_seqs,
    addh + config.BERT_VOCAB_PATH,
    SEQ_LEN=config.SEQ_LEN,
    tag_vocab=tag_vocab
    )

# Build Model

In [0]:
import keras
import keras_bert

In [0]:
bert_model, bert_model_config = keras_bert.build_model_from_config(
    addh + config.BERT_CONFIG_PATH,
    trainable=True
)

In [0]:
input_token = keras.layers.Input(shape=(config.SEQ_LEN,))
input_segment = keras.layers.Input(shape=(config.SEQ_LEN,))

bert_output = bert_model([input_token, input_segment])

output = keras.layers.Dense(len(tag_vocab), activation=keras.activations.softmax)(bert_output)

model = keras.models.Model([input_token, input_segment], output)

In [17]:
model.load_weights(addh + config.MODEL_PATH)

# Predict

In [18]:
predict_tag_id_seqs = model.predict(
    [test_token_id_seqs, test_segment_seqs],
    batch_size=8,
    verbose=1
)

2785/2785 [==============================] - 65s 23ms/step


In [20]:
import analyze
analyze.analyze(
    test_char_seqs,
    test_tag_seqs,
    predict_tag_id_seqs,
    tag_vocab
)

---------------
All: 81966
Wrong: 21372
Wrong Rate: 26%
